In [12]:
import os
import pathlib
import sys
import numpy as np
import pandas as pd
(import matplotlib.pyplot as plt)
from statsmodels.nonparametric.smoothers_lowess import lowess
from datetime import datetime
from pykalman import KalmanFilter
import xml.etree.ElementTree as ET
from math import cos, asin, sqrt, pi

In [78]:
def output_gpx(points, output_filename):
    """
    Output a GPX file with latitude and longitude from the points DataFrame.
    """
    from xml.dom.minidom import getDOMImplementation, parse
    xmlns = 'http://www.topografix.com/GPX/1/0'
    
    def append_trkpt(pt, trkseg, doc):
        trkpt = doc.createElement('trkpt')
        trkpt.setAttribute('lat', '%.10f' % (pt['lat']))
        trkpt.setAttribute('lon', '%.10f' % (pt['lon']))
        time = doc.createElement('time')
        time.appendChild(doc.createTextNode(pt['datetime'].strftime("%Y-%m-%dT%H:%M:%SZ")))
        trkpt.appendChild(time)
        trkseg.appendChild(trkpt)

    doc = getDOMImplementation().createDocument(None, 'gpx', None)
    trk = doc.createElement('trk')
    doc.documentElement.appendChild(trk)
    trkseg = doc.createElement('trkseg')
    trk.appendChild(trkseg)

    points.apply(append_trkpt, axis=1, trkseg=trkseg, doc=doc)

    doc.documentElement.setAttribute('xmlns', xmlns)

    with open(output_filename, 'w') as fh:
        fh.write(doc.toprettyxml(indent='  '))
        
def element_to_data(elem):
    datetime = elem[1].text
    lat = float(elem.get('lat'))
    lon = float(elem.get('lon'))
    return datetime, lat, lon

def get_data(gpx_file):
    parse_result = ET.parse(gpx_file)
    elements = parse_result.iter('{http://www.topografix.com/GPX/1/0}trkpt')
    data= pd.DataFrame(list(map(element_to_data, elements)), columns=['datetime', 'lat', 'lon'])
    data['datetime'] = pd.to_datetime(data['datetime'], utc=True)
    return data

In [99]:
input_directory = 'walk1'
output_directory = 'output'

accl = pd.read_json(input_directory + '/accl.ndjson.gz', lines=True, convert_dates=['timestamp'])[['timestamp', 'x']]
gps = get_data(input_directory + '/gopro.gpx')
phone = pd.read_csv(input_directory + '/phone.csv.gz')[['time', 'gFx', 'Bx', 'By']]

first_time = accl['timestamp'].min()

In [100]:
accl
gps
# phone

,datetime,lat,lon
0,2022-06-08 18:12:09.844000+00:00,49.278656,-123.016106
1,2022-06-08 18:12:09.899142+00:00,49.278609,-123.016093
2,2022-06-08 18:12:09.954285+00:00,49.278635,-123.016249
3,2022-06-08 18:12:10.009428+00:00,49.278625,-123.016099
4,2022-06-08 18:12:10.064571+00:00,49.278669,-123.016219
...,...,...,...
7853,2022-06-08 18:19:22.970998+00:00,49.277593,-123.015828
7854,2022-06-08 18:19:23.026141+00:00,49.277523,-123.015720
7855,2022-06-08 18:19:23.081284+00:00,49.277496,-123.015769
7856,2022-06-08 18:19:23.136427+00:00,49.277516,-123.015880


In [101]:
first_time = accl['timestamp'].min()
phone['timestamp'] = first_time + pd.to_timedelta(phone['time'], unit='sec')
phone['timestamp'] = phone['timestamp'].dt.round(freq='4S')
phone = phone.groupby('timestamp').mean()
phone

,time,gFx,Bx,By
timestamp,,,,
2022-06-08 18:12:08+00:00,0.094858,-0.006360,-11.407500,2.880000
2022-06-08 18:12:12+00:00,2.231188,-0.041580,-18.578124,5.555953
2022-06-08 18:12:16+00:00,6.125778,0.025079,-18.899868,1.150763
2022-06-08 18:12:20+00:00,10.149935,0.060139,-15.033802,-8.815840
2022-06-08 18:12:24+00:00,14.140882,0.080922,-13.939018,-11.510971
...,...,...,...,...
2022-06-08 18:18:52+00:00,402.149981,0.033399,-20.264264,3.550042
2022-06-08 18:18:56+00:00,406.147973,0.053450,-5.007277,15.213546
2022-06-08 18:19:00+00:00,410.151322,0.124351,-0.613833,15.572054


In [102]:
gps['datetime'] = gps['datetime'].dt.round(freq='4S')
gps = gps.groupby('datetime').mean()
gps

,lat,lon
datetime,,
2022-06-08 18:12:08+00:00,49.278633,-123.016149
2022-06-08 18:12:12+00:00,49.278659,-123.016139
2022-06-08 18:12:16+00:00,49.278659,-123.016095
2022-06-08 18:12:20+00:00,49.278638,-123.016024
2022-06-08 18:12:24+00:00,49.278600,-123.015985
...,...,...
2022-06-08 18:19:08+00:00,49.277550,-123.016160
2022-06-08 18:19:12+00:00,49.277542,-123.016076
2022-06-08 18:19:16+00:00,49.277552,-123.015960


In [103]:
accl['timestamp'] = accl['timestamp'].dt.round(freq='4S')
accl = accl.groupby('timestamp').mean()
accl

,x
timestamp,
2022-06-08 18:12:08+00:00,0.008074
2022-06-08 18:12:12+00:00,-0.241065
2022-06-08 18:12:16+00:00,0.401996
2022-06-08 18:12:20+00:00,0.717109
2022-06-08 18:12:24+00:00,0.889633
...,...
2022-06-08 18:19:08+00:00,0.502401
2022-06-08 18:19:12+00:00,0.705140
2022-06-08 18:19:16+00:00,0.644120


In [129]:
best_offset = -5
largest_cross_corr = 0
accl2 = pd.read_json(input_directory + '/accl.ndjson.gz', lines=True, convert_dates=['timestamp'])[['timestamp', 'x']]
first_time = accl2['timestamp'].min()


for offset in np.linspace(-5.0, 5.0, 101):
    phone2 = pd.read_csv(input_directory + '/phone.csv.gz')[['time', 'gFx', 'Bx', 'By']]
    phone2['timestamp'] = first_time + pd.to_timedelta(phone2['time'], unit='sec')
    phone2['timestamp'] = phone2['timestamp'] + pd.to_timedelta(offset, unit='S')
    phone2['timestamp'] = phone2['timestamp'].dt.round(freq='4S')
    phone2 = phone2.groupby('timestamp').mean()
    phone2 = phone2.join(accl)
    phone2 = phone2.dropna()
    
    cross_corr = phone2['gFx']*phone2['x']
    if cross_corr.sum() > largest_cross_corr:
        largest_cross_corr = cross_corr.sum()
        best_offset = offset
    
# phone2['timestamp'] = phone2['timestamp'] + pd.to_timedelta(best_offset, unit='S')
# phone2['timestamp'] = phone2['timestamp'].dt.round(freq='4S')
# phone2 = phone2.groupby('timestamp').mean()
# phone2 = phone2.join(accl)
# phone2 = phone2.dropna()

# cross_corr = phone2['gFx']*phone2['x']
# if cross_corr.sum() > largest_cross_corr:
#     largest_cross_corr = cross_corr.sum()
#     best_offset = offset
print(largest_cross_corr, best_offset)
print(f'Best time offset: {best_offset:.1f}')




3.1419131400557685 -0.5999999999999996
Best time offset: -0.6


In [134]:
combined = phone2.join(gps)
combined['datetime'] = combined.index
combined

,time,gFx,Bx,By,x,lat,lon,datetime
timestamp,,,,,,,,
2022-06-08 18:12:16+00:00,1.677732,-0.026216,-19.223144,6.054855,0.401996,49.278659,-123.016095,2022-06-08 18:12:16+00:00
2022-06-08 18:12:20+00:00,5.123333,-0.009481,-18.081205,3.392026,0.717109,49.278638,-123.016024,2022-06-08 18:12:20+00:00
2022-06-08 18:12:24+00:00,9.154967,0.046496,-17.664085,-7.067517,0.889633,49.278600,-123.015985,2022-06-08 18:12:24+00:00
2022-06-08 18:12:28+00:00,13.152169,0.089435,-11.467221,-12.971721,0.470931,49.278549,-123.015958,2022-06-08 18:12:28+00:00
2022-06-08 18:12:32+00:00,17.159746,0.053194,-16.683908,-8.141972,0.823062,49.278492,-123.015901,2022-06-08 18:12:32+00:00
...,...,...,...,...,...,...,...,...
2022-06-08 18:19:00+00:00,405.153481,0.023981,-9.363223,12.124920,1.334968,49.277485,-123.016246,2022-06-08 18:19:00+00:00
2022-06-08 18:19:04+00:00,409.145368,0.129481,-1.268741,15.244082,1.218110,49.277538,-123.016229,2022-06-08 18:19:04+00:00
2022-06-08 18:19:08+00:00,413.145697,0.117995,-7.607697,12.325872,0.502401,49.277550,-123.016160,2022-06-08 18:19:08+00:00
